In [1]:
# -*- coding:UTF-8 -*-
import requests        #导入requests包
from bs4 import    BeautifulSoup
import pymysql
import multiprocessing
from multiprocessing import Pool
import pandas as pd
import time
import random

In [60]:
db = pymysql.connect(host='localhost',user='root',password='ky@19961012',db='mysql',port=3306,charset='utf8')
conn = db.cursor() #获取指针以操作数据库
conn.execute('set names utf8')

/opt/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


0

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
           'cookie': 'wanplus_token=0191eeff90ac0b88c612a8302ad6186c; wanplus_storage=lf4m67eka3o; wanplus_sid=d10581e1b9734cddb08055b7d921eb03; wanplus_csrf=_csrf_tk_1746654912; gameType=2; UM_distinctid=173c9976a77bf-023ac10de36a4-31647305-fa000-173c9976a784a9; wp_pvid=4468120896; wp_info=ssid=s6393116652; Hm_lvt_f69cb5ec253c6012b2aa449fb925c1c2=1596815011; CNZZDATA1275078652=2099498220-1596809682-%7C1596961256; Hm_lpvt_f69cb5ec253c6012b2aa449fb925c1c2=1596964168',
           'x-csrf-token': '1763432128',
           'x-requested-with': 'XMLHttpRequest', 
           'Accept':'application/json, text/javascript, */*; q=0.01',
           'Accept-Encoding':'gzip, deflate,  br',
           'Accept-Language':'zh-CN,zh;q=0.9',
           'Connection':'keep-alive',}

target = 'https://www.wanplus.com/ajax/matchdetail/69233?_gtk=1763432128'
req = requests.get(url = target,headers=headers)



KeyboardInterrupt: 

In [46]:
'''
#解析单场比赛数据
'''
def parse_page_index(html):
    try:
        data = json.loads(html)
        for i in range(2):
            if i == 0:
                color = 'blue'
            else:
                color = 'red'
            for j in range(5):
                x = str(j + 1)
                json1 = data.get('data').get('plList')[i].get(x)
                json2 = json1.get('stats')
                json3 = data.get('data').get('info')

                gameversion = json3.get('gameversion')  # 游戏版本
                blueid = json3.get('oneteam').get('teamid')  # 蓝色方队伍id
                bluename = json3.get('oneteam').get('teamalias')  # 蓝色方队伍名称
                redid = json3.get('twoteam').get('teamid')  # 红色方队伍id
                redname = json3.get('twoteam').get('teamalias')  # 红色方队伍名称
                duration = json3.get('duration')  # 游戏时长
                matchorder = json3.get('matchorder')  # 游戏场次
                winid = json3.get('winner')  # 胜利方队伍id
                gametype = json3.get('gametype')  # 比赛类型bo几

                baron1 = data.get('data').get('teamStatsList').get('baronkills')[0]  # 蓝色方男爵击杀次数
                baron2 = data.get('data').get('teamStatsList').get('baronkills')[1]  # 红色方男爵击杀次数
                dragon1 = data.get('data').get('teamStatsList').get('dragonkills')[0]  # 蓝色方小龙击杀次数
                dragon2 = data.get('data').get('teamStatsList').get('dragonkills')[1]  # 红色方小龙击杀次数

                scheduleid = data.get('data').get('plList')[0].get('1').get('scheduleid')  # 比赛id
                matchid = data.get('data').get('plList')[0].get('1').get('matchid')  # 单场id

                ban = data.get('data').get('bpList').get('bans')[i][j].get('cpherokey')  # ban人
                pick = json1.get('cpherokey')  # 选择英雄

                lane = json1.get('lane')  # 位置
                kda = json1.get('kda')  # kda
                playername = json1.get('playername')  # 选手名称
                playerid = json1.get('playerid')  # 选手id

                assists = json2.get('assists')  # 助攻数
                deaths = json2.get('deaths')  # 死亡数
                doubleKills = json2.get('doubleKills')  # 双杀次数
                firstBloodAssist = json2.get('firstBloodAssist')  # 是否助攻一血
                firstBloodKill = json2.get('firstBloodKill')  # 是否拿下一血
                firstTowerAssist = json2.get('firstTowerAssist')  # 是否助攻一塔
                firstTowerKill = json2.get('firstTowerKill')  # 是否拿下一塔
                goldEarned = json2.get('goldEarned')  # 金钱数
                goldSpent = json2.get('goldSpent')  # 花钱数
                killingSprees = json2.get('killingSprees')  # 杀人如麻次数
                kills = json2.get('kills')  # 击杀数
                largestKillingSpree = json2.get('largestKillingSpree')  # 最大杀人如麻次数
                largestMultiKill = json2.get('largestMultiKill')  # 最大连杀数
                magicDamageDealt = json2.get('magicDamageDealt')  # 造成的魔法伤害
                magicDamageDealtToChampions = json2.get('magicDamageDealtToChampions')  # 对英雄造成的魔法伤害
                magicDamageTaken = json2.get('magicDamageTaken')  # 承受的魔法伤害
                minionsKilled = json2.get('minionsKilled')  # 补刀数
                neutralMinionsKilledEnemyJungle = json2.get('neutralMinionsKilledEnemyJungle')  # 敌方野怪击杀数
                neutralMinionsKilledTeamJungle = json2.get('neutralMinionsKilledTeamJungle')  # 己方野怪击杀数
                physicalDamageDealt = json2.get('physicalDamageDealt')  # 造成的物理伤害
                physicalDamageDealtToChampions = json2.get('physicalDamageDealtToChampions')  # 对英雄造成的物理伤害
                physicalDamageTaken = json2.get('physicalDamageTaken')  # 承受的物理伤害
                totalDamageDealt = json2.get('totalDamageDealt')  # 造成的总伤害
                totalDamageDealtToChampions = json2.get('totalDamageDealtToChampions')  # 对英雄造成的总伤害
                totalDamageTaken = json2.get('totalDamageTaken')  # 承受的总伤害
                totalHeal = json2.get('totalHeal')  # 治疗量
                trueDamageDealt = json2.get('trueDamageDealt')  # 造成的真实伤害
                trueDamageDealtToChampions = json2.get('trueDamageDealtToChampions')  # 对英雄造成的真实伤害
                trueDamageTaken = json2.get('trueDamageTaken')  # 承受的真实伤害
                items = {
                    'gameversion': gameversion,
                    'blueid': blueid,
                    'bluename': bluename,
                    'redid': redid,
                    'redname': redname,
                    'duration': duration,
                    'winid': winid,
                    'gametype': gametype,
                    'scheduleid': scheduleid,
                    'matchid': matchid,
                    'baron1': baron1,
                    'baron2': baron2,
                    'dragon1': dragon1,
                    'dragon2': dragon2,
                    'color': color,
                    'ban': ban,
                    'x': x,
                    'pick': pick,
                    'lane': lane,
                    'kda': kda,
                    'playername': playername,
                    'playerid': playerid,
                    'assists': assists,
                    'deaths': deaths,
                    'doubleKills': doubleKills,
                    'firstBloodAssist': firstBloodAssist,
                    'firstBloodKill': firstBloodKill,
                    'firstTowerAssist': firstTowerAssist,
                    'firstTowerKill': firstTowerKill,
                    'goldEarned': goldEarned,
                    'goldSpent': goldSpent,
                    'killingSprees': killingSprees,
                    'kills': kills,
                    'largestKillingSpree': largestKillingSpree,
                    'largestMultiKill': largestMultiKill,
                    'magicDamageDealt': magicDamageDealt,
                    'magicDamageDealtToChampions': magicDamageDealtToChampions,
                    'magicDamageTaken': magicDamageTaken,
                    'minionsKilled': minionsKilled,
                    'neutralMinionsKilledEnemyJungle': neutralMinionsKilledEnemyJungle,
                    'neutralMinionsKilledTeamJungle': neutralMinionsKilledTeamJungle,
                    'physicalDamageDealt': physicalDamageDealt,
                    'physicalDamageDealtToChampions': physicalDamageDealtToChampions,
                    'physicalDamageTaken': physicalDamageTaken,
                    'totalDamageDealt': totalDamageDealt,
                    'totalDamageDealtToChampions': totalDamageDealtToChampions,
                    'totalDamageTaken': totalDamageTaken,
                    'totalHeal': totalHeal,
                    'trueDamageDealt': trueDamageDealt,
                    'trueDamageDealtToChampions': trueDamageDealtToChampions,
                    'trueDamageTaken': trueDamageTaken
                }
                #save_to_mongo(items)
                print(items)
    except json.JSONDecodeError :
        print('JSONDecodeError')
        pass

In [47]:
parse_page_index(req.text)


{'gameversion': '10.15.1', 'blueid': '4043', 'bluename': 'JDG', 'redid': '63', 'redname': 'OMG', 'duration': '29:04', 'winid': '4043', 'gametype': '2', 'scheduleid': '63564', 'matchid': '69233', 'baron1': '1', 'baron2': '0', 'dragon1': '1', 'dragon2': '2', 'color': 'blue', 'ban': 'Azir', 'x': '1', 'pick': '迷失之牙', 'lane': 'MID', 'kda': '8', 'playername': 'Zoom', 'playerid': '18838', 'assists': 5, 'deaths': 0, 'doubleKills': 0, 'firstBloodAssist': False, 'firstBloodKill': False, 'firstTowerAssist': False, 'firstTowerKill': False, 'goldEarned': 12565, 'goldSpent': 10275, 'killingSprees': 1, 'kills': 3, 'largestKillingSpree': 3, 'largestMultiKill': 1, 'magicDamageDealt': 16380, 'magicDamageDealtToChampions': 3502, 'magicDamageTaken': 4064, 'minionsKilled': 247, 'neutralMinionsKilledEnemyJungle': 0, 'neutralMinionsKilledTeamJungle': 5, 'physicalDamageDealt': 153974, 'physicalDamageDealtToChampions': 15174, 'physicalDamageTaken': 12633, 'totalDamageDealt': 170380, 'totalDamageDealtToChampion

In [39]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
           'cookie': 'wanplus_token=0191eeff90ac0b88c612a8302ad6186c; wanplus_storage=lf4m67eka3o; wanplus_sid=d10581e1b9734cddb08055b7d921eb03; wanplus_csrf=_csrf_tk_1746654912; gameType=2; UM_distinctid=173c9976a77bf-023ac10de36a4-31647305-fa000-173c9976a784a9; wp_pvid=4468120896; wp_info=ssid=s6393116652; Hm_lvt_f69cb5ec253c6012b2aa449fb925c1c2=1596815011; CNZZDATA1275078652=2099498220-1596809682-%7C1596961256; Hm_lpvt_f69cb5ec253c6012b2aa449fb925c1c2=1596964168',
           'x-csrf-token': '1763432128',
           'x-requested-with': 'XMLHttpRequest', 
           'Accept':'application/json, text/javascript, */*; q=0.01',
           'Accept-Encoding':'gzip, deflate,  br',
           'Accept-Language':'zh-CN,zh;q=0.9',
           'Connection':'keep-alive',}

target = 'https://www.wanplus.com/ajax/matchdetail/70003?_gtk=1763432128'
req = requests.get(url = target,headers=headers)



<Response [200]>

In [4]:

'''打开单场比赛索引页'''
def get_page_index(url):
    try:
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except requests.RequestException:
        #print('请求索引页出错')
        return None
'''
#解析单场比赛数据
'''
def parse_page_index1(html):
    try:
        dat=[]
        data = json.loads(html)
        for i in range(2):
            if i == 0:
                color = 'blue'
            else:
                color = 'red'
            for j in range(5):
                x = str(j + 1)
                json1 = data.get('data').get('plList')[i].get(x)
                json2 = json1.get('stats')
                json3 = data.get('data').get('info')
                pick = json1.get('cpherokey')  # 选择英雄

                items = {
                    'pick': pick,

                }
                dat.append(pick)
                
                #save_to_mongo(items)
                #print(items)
        return dat
    except json.JSONDecodeError :
        print('JSONDecodeError')
        pass
    
'''主函数'''  
def main(num):
        print(num)
        random.random()
        html = get_page_index('https://www.wanplus.com/ajax/matchdetail/'+str(num)+'?_gtk=1763432128')
        try:
            a1= parse_page_index1(html)
            #print(a1)
            res.append(a1)
        except AttributeError:
            pass
        except IndexError:
            pass
        except TypeError:
            pass

In [45]:

if __name__ == '__main__':
    res=multiprocessing.Manager().list([])
    pool = Pool()
    pool.map(main, [i for i in range(1,5000)])#66914,68334
    print('结束啦！')
    pool.close()
    pool.join()
    df = pd.DataFrame(list(res), columns=['t11', 't12', 't13','t14','t15','t21', 't22', 't23','t24','t25']) 

1314627

940



Process ForkPoolWorker-69:
Process ForkPoolWorker-67:


KeyboardInterrupt: 

Process ForkPoolWorker-68:
Process ForkPoolWorker-66:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-4-f0924dcbec18>", line 47, i

12531566

1879


Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-4-f0924dcbec18>", line 47, in main
    html = get_page_index('https://www.wanplus.com/ajax/matchdetail/'+str(num)+'?_gtk=1763432128')
  File "<ipython-input-4-f0924dcbec18>", line 4, in get_page_index
    response = requests.get(url,headers=headers)
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/api.py", line 76, in get
    return request('get', url, params=params, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/api.py", line 61, in re

2192
1254
15671880

2193
125515681881


2194
1256
15691882

2195
1257
1570
1883
2196
1258
18841571

2197
1259
15721885

2198
1260
15731886

2199
1261
18871574

2200
1262
1575
1888
2201
1263
1576
1889
2202
1264
15771890

2203
189112651578


2204
189212661579


2205
189315801267


2206
126818941581


2207
126915821895


2208
158318961270


2209
189712711584


2210
189815851272


2211
189912731586


2212
190015871274


2213
1588190112752214



1902158922151276



2216159012771903



1904127815912217



2218127915921905



1593221912801906



1281222015941907



159519082221

1282

1596
1909
2222
1283
2223159712841910



1598191112852224



1912128622251599



1600222612871913



1288191422271601



1915222816021289



2229160312901916



22301291
16041917


2231160512921918



2232129316061919



1607129422331920



1921223412951608



1609223519221296



223616101923
1297


192422371611

1298

19252238

12991612

22391926

13001613

2240
1927
13011614

2241
1928
16151302

22421929

16161

2929
2617
2930
2618
2931
2619
2932
2933
2620
2621
2934
2622
2935
2623
2936
2624
2937
2625
2938
2626
2939
2940
2627
2941
2942
2628
26292943

2944
2630
2945
2631
2632
2946
2633
2947
2634
2948
2635
2949
2636
2950
2637
2951
26382952

2639
2953
26402954

2641
2955
2642
2956
26432957

2644
2958
2645
2959
2646
2960
2647
2961
2648
2962
2649
2963
2650
2964
2651
2965
2652
2966
2653
2654
2967
2968
2655
29692656

2970
2657
2971
2658
2972
2659
2973
2660
2974
2661
2662
2975
2976
2663
2977
2664
2978
2665
2979
2666
2980
2667
2981
2668
2982
2669
2983
2670
2984
2671
2985
2672
2986
2673
2987
2674
2988
2675
2989
26762990

2677
2991
2678
2992
2679
2993
2680
2994
2681
2995
2682
2996
2683
2997
2998
2684
2999
2685
3000
2686
3001
2687
3002
2688
3003
2689
3004
2690
3005
2691
3006
2692
3007
2693
3008
3009
2694
3010
2695
3011
2696
2697
3012
2698
3013
2699
3014
2700
3015
2701
3016
2702
3017
2703
27043018

27053019

3020
2706
3021
2707
2708
3022
2709
3023
2710
3024
2711
3025
2712
3026
2713
3027
2714
3028
2715
3029


In [41]:
df

,t11,t12,t13,t14,t15,t21,t22,t23,t24,t25


In [142]:

df.to_csv('LPL夏季赛.csv',sep=',',index=False,header=True)


In [30]:
df1_10000=df

In [36]:
df10000_20000=df

In [8]:
df20000_25000=df

In [11]:
df25000_30000=df

In [15]:
df30000_35000=df

In [18]:
df35000_40000=df

In [22]:
df40000_45000=df

In [25]:
df45000_50000=df

In [28]:
df50000_55000=df

In [32]:
df55000_60000=df

In [35]:
df60000_65000=df

In [38]:
df65000_70000=df

In [47]:
frames = [df20000_25000,df25000_30000,df30000_35000,df35000_40000,df40000_45000,df45000_50000,df50000_55000,df55000_60000,df60000_65000,df65000_70000]

result = pd.concat(frames)

In [44]:
1-20000   有问题

In [48]:
result

,t11,t12,t13,t14,t15,t21,t22,t23,t24,t25
0,Swain,Graves,Azir,Sivir,Braum,Fizz,RekSai,Lissandra,Vayne,Karma
1,Trundle,Kindred,TwistedFate,Caitlyn,Bard,Rumble,Elise,Karma,Lucian,Braum
2,Trundle,Nidalee,Taliyah,Ezreal,Karma,Maokai,LeeSin,Leblanc,Sivir,Braum
3,Trundle,RekSai,Varus,Sivir,Nami,Maokai,Graves,Viktor,Lucian,Karma
4,Jax,RekSai,Lissandra,Lucian,Braum,Irelia,Gragas,Viktor,Sivir,Karma
...,...,...,...,...,...,...,...,...,...,...
3095,Sylas,Khazix,Pantheon,Caitlyn,Thresh,Ornn,Shyvana,Jayce,Ezreal,Karma
3096,Sion,Sett,Leblanc,Ashe,Nautilus,Ornn,Olaf,Syndra,Caitlyn,Bard
3097,Aatrox,LeeSin,Azir,Ashe,Karma,Gangplank,Olaf,Syndra,Caitlyn,Bard
3098,Gangplank,Trundle,Azir,Tristana,Karma,Malphite,Graves,Orianna,Ezreal,Bard


In [50]:
result.to_csv('LOL数据.csv',sep=',',index=False,header=True)